In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model
from sklearn import metrics
import seaborn as sns 
import math
import threading
%matplotlib inline

In [61]:
df = pd.read_csv('original.csv')
df['DATE'] = pd.to_datetime(df['INSPECTION DATE'])
a = df['DATE'].min()
df[df['DATE'] == a] = np.nan
df.sort_values(['DATE','CAMIS'],ascending = [0,0],inplace = True)
df['CRITICAL FLAG'] = df['CRITICAL FLAG'].map({'Critical': 1, 'Not Critical': 0,'Not Applicable':0})
df.drop(['BUILDING','STREET','BORO','ZIPCODE','PHONE','RECORD DATE','INSPECTION DATE','GRADE DATE','INSPECTION TYPE'],axis=1,inplace=True)
df['CAMIS2'] = df['CAMIS']
df['DATE2'] = df['DATE']

In [63]:
def sericoncat(seri):
    return seri.str.cat()
def getfirst(seri):
    return seri.iloc[0]

df = df.groupby(['CAMIS','DATE']).agg({'CRITICAL FLAG': np.sum, 'SCORE': np.mean,'GRADE': getfirst})
#df = df[df['CAMIS2'].isnull() == False]
#lstcritcal date critical num critical type

df['last_crit_flag'] = np.nan
df['last_violation_code'] = np.nan
df['last_score'] = np.nan
df['last_inspection_date'] = np.nan

KeyError: 'CAMIS2'

In [65]:
df['TARGET'] = np.where(df['CRITICAL FLAG']>0, 1, 0)
df['TARGET'].value_counts()

1    114160
0     17987
Name: TARGET, dtype: int64

In [69]:
17987/len(df)

0.13611357049346562

In [66]:
len(df)

132147

In [ ]:
def buildPrevInfo(df,start, end):
    if start == 0:
        prev = df.iloc[len(df)-1,:]
    else:
        prev = df.iloc[start-1,:]
    for i in range (start,end):
        if(i%100 == 0):
            print("Thread-{}: Querying {}th entry\n".format(start, i))
        curr = df.iloc[i,:]
        if prev['CAMIS2']==curr['CAMIS2']:
            # add prev info to curr row
            df.iloc[i,8] = str(prev['CRITICAL FLAG'])
            df.iloc[i,9] = str(prev['VIOLATION CODE'])
            df.iloc[i,10] = str(prev['SCORE'])
            df.iloc[i, 11] = str(prev['DATE2'])
        prev = curr
buildPrevInfo(df,0,len(df))

In [ ]:
df['TARGET'] = np.where(df['CRITICAL FLAG']>0, 1, 0)
df['TARGET'].value_counts()

In [ ]:
# Find a date to set as time_start
# for i in range(100):
#     print('{}th row has date {}'.format(i,df['DATE2'][i]))
# GET "8th row has date 2017-06-26 00:00:00"
time_start = df['DATE2'][8]

In [ ]:
# Since there are more values wit target val = 1 than 0
# Selectively drop some inbalanced data that met the following criteria
df['TARGET'] = np.where(df['CRITICAL FLAG']>0, 1, 0)

#df['TARGET'].value_counts() # get ratio for splitting test set

#Drop if we could not get cooresponding yelp data for the record
tmp = pd.read_csv('matched_yelp_data.csv')
camisset = set(tmp['CAMIS'].unique())
df = df[df['CAMIS2'].isin(camisset)]

#Drop if TARGET val is 1 and no previous data
df = df.drop(df[(df['TARGET']==1)&df['last_crit_flag'].isnull()].index)

# Drop all records before 2017-01-01 if TARGET val is 1 and more than 4 entries for this restaurant exists in the dataframe
tmp = (df['CAMIS2'].value_counts()>4).to_frame()
tmp2 = set(tmp[tmp['CAMIS2'] == True].index.tolist())
df = df.drop(df[(df['TARGET'] == 1)& (~df['CAMIS2'].isin(tmp2)) & (df['DATE2']<time_start)].index)

In [ ]:
# Re_Index
df.sort_values(['CAMIS2','DATE2'],ascending = [1,0],inplace = True)
df.reset_index(inplace=True)
df['index'] = df.index
original = df
original #Keep original for record

In [57]:
ratio

NameError: name 'ratio' is not defined

In [ ]:
# Create test set so that it contains 500 data with TARGET = 1 and 500 data with TARGET = 0
tmp = df.groupby(['CAMIS2']).agg({'index': getfirst,'CRITICAL FLAG': getfirst,'VIOLATION CODE':getfirst, 'ACTION':getfirst, 'VIOLATION DESCRIPTION':getfirst, 'SCORE': getfirst,'CUISINE DESCRIPTION': getfirst, 'CAMIS2':getfirst, 'DATE2':getfirst,'last_crit_flag':getfirst,'last_violation_code':getfirst, 'last_score':getfirst, 'last_inspection_date': getfirst, 'TARGET':getfirst})
tar_0 = tmp[tmp['TARGET'] > 0].sample(n=int(1000*ratio))
tar_1 = tmp[tmp['TARGET'] == 0].sample(n=1000-int(1000*ratio))
testset = pd.concat([tar_0,tar_1])
test_index_set = set(testset['index'].values)
training_base = df[~df['index'].isin(test_index_set)]

In [68]:
aaa = pd.read_csv('01_CAMIS_TRAINING__WITH_PREV.csv')
len(aaa)

65886

In [ ]:
testset.to_csv('01_CAMIS_BALANCED_TESTSET.csv')
training_base.to_csv('01_CAMIS_BALANCED_TRAINING.csv')
original.to_csv('01_CAMIS_FULL_WITH_PREV.csv')
print('Test Set Info:\n{}\n'.format(testset['TARGET'].value_counts()))
print('Training Set Info:\n{}'.format(training_base['TARGET'].value_counts()))